In [3]:
import pandas as pd
import numpy as np


In [4]:
df=pd.read_csv("C:\\Users\\saipr\\Downloads\\Combined Data.csv (1)\\Combined Data.csv")

In [5]:
df

,Unnamed: 0,statement,status
0,0,oh my gosh,Anxiety
1,1,"trouble sleeping, confused mind, restless hear...",Anxiety
2,2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety
3,3,I've shifted my focus to something else but I'...,Anxiety
4,4,"I'm restless and restless, it's been a month n...",Anxiety
...,...,...,...
53038,53038,Nobody takes me seriously I’ve (24M) dealt wit...,Anxiety
53039,53039,"selfishness ""I don't feel very good, it's lik...",Anxiety
53040,53040,Is there any way to sleep better? I can't slee...,Anxiety
53041,53041,"Public speaking tips? Hi, all. I have to give ...",Anxiety


In [6]:
df.isnull().sum()

Unnamed: 0      0
statement     362
status          0
dtype: int64

In [7]:
df.dropna(subset='statement')

,Unnamed: 0,statement,status
0,0,oh my gosh,Anxiety
1,1,"trouble sleeping, confused mind, restless hear...",Anxiety
2,2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety
3,3,I've shifted my focus to something else but I'...,Anxiety
4,4,"I'm restless and restless, it's been a month n...",Anxiety
...,...,...,...
53038,53038,Nobody takes me seriously I’ve (24M) dealt wit...,Anxiety
53039,53039,"selfishness ""I don't feel very good, it's lik...",Anxiety
53040,53040,Is there any way to sleep better? I can't slee...,Anxiety
53041,53041,"Public speaking tips? Hi, all. I have to give ...",Anxiety


In [8]:
df['status'].value_counts()

status
Normal                  16351
Depression              15404
Suicidal                10653
Anxiety                  3888
Bipolar                  2877
Stress                   2669
Personality disorder     1201
Name: count, dtype: int64

In [9]:
X=df['statement']
y=df['status']

In [10]:
import re
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Ensure required NLTK data is available
import nltk
nltk.download('stopwords')
nltk.download('punkt')

# Load stopwords only once
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\saipr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\saipr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
def preprocess_text(text):
    if isinstance(text, str):  # Check if text is a string
        # Lowercase and remove punctuation
        text = re.sub(r'[^\w\s]', '', text.lower())
        # Tokenize
        tokens = word_tokenize(text)
        # Remove stopwords
        tokens = [word for word in tokens if word not in stop_words]
        return ' '.join(tokens)
    else:
        return ''  # Return an empty string for non-string entries

# Apply preprocessing
X = df['statement'].apply(preprocess_text)


In [12]:
from sklearn.model_selection import train_test_split  # Import the function

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer  # Import TfidfVectorizer

# Convert text data to numerical data using TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [15]:
from sklearn.linear_model import LogisticRegression  # Import LogisticRegression

# Train a logistic regression model

model = LogisticRegression(max_iter=1000,random_state=42)
model.fit(X_train_tfidf, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [16]:
# Make predictions on the test set
y_pred = model.predict(X_test_tfidf)


In [17]:
from sklearn.metrics import accuracy_score, classification_report  # Import evaluation metrics

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.7590724856254124
Classification Report:
                       precision    recall  f1-score   support

             Anxiety       0.82      0.77      0.79       779
             Bipolar       0.87      0.68      0.76       580
          Depression       0.71      0.73      0.72      3100
              Normal       0.82      0.95      0.88      3327
Personality disorder       0.87      0.42      0.57       248
              Stress       0.76      0.45      0.56       557
            Suicidal       0.67      0.64      0.65      2018

            accuracy                           0.76     10609
           macro avg       0.79      0.66      0.71     10609
        weighted avg       0.76      0.76      0.75     10609



In [47]:
# Define the new review
new_review = ""

preprocessed_input = preprocess_text(new_review)


# Convert the new review to numerical format using the previously fitted TF-IDF vectorizer
new_review_tfidf = vectorizer.transform([preprocessed_input])

# Predict the sentiment using the trained model
prediction = model.predict(new_review_tfidf)

# Output the prediction
print("Prediction (1 = Liked, 0 = Not Liked):", prediction[0])


Prediction (1 = Liked, 0 = Not Liked): Normal


In [20]:
import joblib

# Save the model and vectorizer
joblib.dump(model, 'model.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')  # Save the vectorizer


['vectorizer.pkl']

In [22]:
print(model)
print(vectorizer)


LogisticRegression(max_iter=1000, random_state=42)
TfidfVectorizer(stop_words='english')
